In [ ]:
from fastai.vision.all import *
import librosa

From https://www.kaggle.com/kneroma/clean-fast-simple-bird-identifier-inference getting file locations and labels in case of train:

In [ ]:
NUM_CLASSES = 397
SR = 32_000
DURATION = 5
THRESH = 0.25
TEST_AUDIO_ROOT = Path("../input/birdclef-2021/test_soundscapes")
SAMPLE_SUB_PATH = "../input/birdclef-2021/sample_submission.csv"
TARGET_PATH = None
if not len(list(TEST_AUDIO_ROOT.glob("*.ogg"))):
    TEST_AUDIO_ROOT = Path("../input/birdclef-2021/train_soundscapes")
    SAMPLE_SUB_PATH = None
    # SAMPLE_SUB_PATH = "../input/birdclef-2021/sample_submission.csv"
    TARGET_PATH = Path("../input/birdclef-2021/train_soundscape_labels.csv")

In [ ]:
data = pd.DataFrame(
     [(path.stem, *path.stem.split("_"), path) for path in Path(TEST_AUDIO_ROOT).glob("*.ogg")],
    columns = ["filename", "id", "site", "date", "filepath"]
)
print(data.shape)
data.head()

In [ ]:
df_train = pd.read_csv("../input/birdclef-2021/train_metadata.csv")

LABEL_IDS = {label: label_id for label_id,label in enumerate(sorted(df_train["primary_label"].unique()))}
INV_LABEL_IDS = {val: key for key,val in LABEL_IDS.items()}
df_train.head()

In [ ]:
row_ids = []
filenames = []
for fn, sid, site, date, filepath in data.values:
    y, sr = librosa.load(filepath, sr=32000)
    n_clips = len(y)/(32000*5)
    for c in range(int(n_clips)):
        row_ids.append('_'.join(fn.split('_')[:-1]) + '_' + str(5*(c+1)))
        filenames.append(filepath)
submission_df = pd.DataFrame({
    'row_id':row_ids,
    'birds':['nocall' for _ in row_ids],
    'fn':filenames,
    'start_time':[int(r.split('_')[-1])-5 for r in row_ids]
})
print(submission_df.shape)
submission_df.head()

# Making Preds

Bla bla threshold etc

In [ ]:
def chunk_to_spec(chunk, SPEC_HEIGHT=64,SPEC_WIDTH=256, rate=32000, FMIN=200, FMAX=12500):
    mel_spec = librosa.feature.melspectrogram(y=chunk, 
                                              sr=32000, 
                                              n_fft=1024, 
                                              hop_length=int(32000 * 5 / (SPEC_WIDTH - 1)), 
                                              n_mels=SPEC_HEIGHT, 
                                              fmin=FMIN, 
                                              fmax=FMAX)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec_db

class TitledImage(fastuple):
    def show(self, ctx=None, **kwargs): show_titled_image(self, ctx=ctx, **kwargs)

class ClipTransform(ItemTransform):

    def __init__(self, df):
        self.df=df
        self.vocab,self.o2i = uniqueify(df['label'], sort=True, bidir=True)
        
    def encodes(self, i, from_np=False):
        row_id, birds, fn, start_time, label = self.df.iloc[i].values
        y, sr = librosa.load(fn, sr=32000, offset=start_time, duration=5)
        spec = chunk_to_spec(y,SPEC_HEIGHT=112,SPEC_WIDTH=224)
        spec -= np.min(spec) 
        spec /= 80 # np.max(spec) # Normalize
        spec =  torch.unsqueeze(tensor(spec), 0)
        spec = torch.cat([spec, spec, spec]) # Stack three channels to simulate RGB if using a pretrained model
        return spec, self.o2i[label]
    
    def decodes(self, x):
        return TitledImage(x[0],self.vocab[x[1]])


df_small = submission_df
df_small['label'] = 'LABEL'
clip_tfm = ClipTransform(df_small)

# Not using either of these apart from making the dataloaders so can ignore the obvious issues
train =  df_small.sample(frac=0.1)
valid =  df_small.sample(frac=0.1)
train_idx, valid_idx = list(train.index), list(valid.index)
print('train and val size', len(train_idx), len(valid_idx))
train_tl= TfmdLists(train_idx, clip_tfm)
valid_tl= TfmdLists(valid_idx, clip_tfm)
dls = DataLoaders.from_dsets(train_tl, valid_tl, bs=16)
dls = dls.cuda()
test_dl = dls.test_dl(df_small.index)
xb, yb = dls.one_batch()
print(xb.shape)

In [ ]:
learn = load_learner('../input/baseline-model/baseline_3e.pkl')

In [ ]:
preds = learn.get_preds(dl=test_dl)

In [ ]:
preds[0].shape

In [ ]:
bird_names = df_train.primary_label.unique()

In [ ]:
pred_calls = []
thresh = 0.45
for p in preds[0]:
    calls = []
    for i, prob in enumerate(p):
        if prob > thresh:
            calls.append(bird_names[i])
    if len(calls)==0:
        calls.append('nocall')
    pred_calls.append(calls)
sub = submission_df[['row_id', 'birds']].copy()
sub['birds'] = [' '.join(calls) for calls in pred_calls]
sub.sample(10)

In [ ]:
sub.to_csv("submission.csv", index=False)

In [ ]:
def get_metrics(s_true, s_pred):
    s_true = set(s_true.split())
    s_pred = set(s_pred.split())
    n, n_true, n_pred = len(s_true.intersection(s_pred)), len(s_true), len(s_pred)
    
    prec = n/n_pred
    rec = n/n_true
    f1 = 2*prec*rec/(prec + rec) if prec + rec else 0
    
    return {"f1": f1, "prec": prec, "rec": rec, "n_true": n_true, "n_pred": n_pred, "n": n}
if TARGET_PATH:
    sub_target = pd.read_csv(TARGET_PATH)
    sub_target = sub_target.merge(sub, how="left", on="row_id")
    
    print(sub_target["birds_x"].notnull().sum(), sub_target["birds_x"].notnull().sum())
    assert sub_target["birds_x"].notnull().all()
    assert sub_target["birds_y"].notnull().all()
    
    df_metrics = pd.DataFrame([get_metrics(s_true, s_pred) for s_true, s_pred in zip(sub_target.birds_x, sub_target.birds_y)])
    
    print(df_metrics.mean())

In [ ]:
# sub_target[sub_target.birds_y != "nocall"]

In [ ]:
# Exploring the threshold - anything between 0.15 and 0.6 does about the same, peaking around 0.4 in this case.
# for thresh in range(5, 95, 5):
#     pred_calls = []
#     thresh = thresh/100.0
#     for p in preds[0]:
#         calls = []
#         for i, prob in enumerate(p):
#             if prob > thresh:
#                 calls.append(bird_names[i])
#         if len(calls)==0:
#             calls.append('nocall')
#         pred_calls.append(calls)
#     sub = submission_df[['row_id', 'birds']].copy()
#     sub['birds'] = [' '.join(calls) for calls in pred_calls]
    
#     sub_target = pd.read_csv(TARGET_PATH)
#     sub_target = sub_target.merge(sub, how="left", on="row_id")
    
#     print(sub_target["birds_x"].notnull().sum(), sub_target["birds_x"].notnull().sum())
#     assert sub_target["birds_x"].notnull().all()
#     assert sub_target["birds_y"].notnull().all()
    
#     df_metrics = pd.DataFrame([get_metrics(s_true, s_pred) for s_true, s_pred in zip(sub_target.birds_x, sub_target.birds_y)])
    
#     print(df_metrics.mean())